In [1]:
from datetime import datetime

In [2]:
current = datetime.utcnow()

In [3]:
current

datetime.datetime(2024, 12, 31, 5, 55, 40, 256002)

In [4]:
import json

In [5]:
json.dumps(current)

TypeError: Object of type datetime is not JSON serializable

In [6]:
str(current)

'2024-12-31 05:55:40.256002'

In [9]:
def format_iso(dt):
    return dt.strftime('%Y-%m-%dT%H:%M:%S')

In [10]:
format_iso(current)

'2024-12-31T05:55:40'

In [11]:
current.isoformat()

'2024-12-31T05:55:40.256002'

In [12]:
log_record = {
    'time': datetime.utcnow().isoformat(),
    'message': 'Hello, world!'
}
json.dumps(log_record)

'{"time": "2024-12-31T06:01:58.915733", "message": "Hello, world!"}'

In [13]:
log_record = {
    'time': datetime.utcnow(),
    'message': 'Hello, world!'
}
json.dumps(log_record, default=format_iso)


'{"time": "2024-12-31T06:04:29", "message": "Hello, world!"}'

In [14]:
def format_general(arg):
    return 'Unknown serializion'

In [15]:
json.dumps(log_record, default=format_general)

'{"time": "Unknown serializion", "message": "Hello, world!"}'

In [16]:
log_record = {
    'time': datetime.utcnow(),
    'message': 'Hello, world!',
    'args': {10, "test"}
}
json.dumps(log_record, default=format_general)


'{"time": "Unknown serializion", "message": "Hello, world!", "args": "Unknown serializion"}'

In [17]:
log_record = {
    'time1': datetime.utcnow(),
    'time2': datetime.utcnow(),
    'message': 'Hello, world!'
}
json.dumps(log_record, default=format_general)


'{"time1": "Unknown serializion", "time2": "Unknown serializion", "message": "Hello, world!"}'

In [18]:
json.dumps(log_record, default=format_iso)

'{"time1": "2024-12-31T06:07:01", "time2": "2024-12-31T06:07:01", "message": "Hello, world!"}'

In [20]:
log_record = {
    'time': datetime.utcnow(),
    'message': 'Hello, world!',
    'args': {10, "test"}
}
json.dumps(log_record, default=format_iso)



AttributeError: 'set' object has no attribute 'strftime'

In [22]:
def custom_json_formatter(arg):
    if isinstance(arg, datetime):
        return arg.isoformat()
    elif isinstance(arg, set):
        return list(arg)
    return str(arg)

json.dumps(log_record, default=custom_json_formatter)

'{"time": "2024-12-31T06:08:00.148441", "message": "Hello, world!", "args": [10, "test"]}'

In [23]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        self.create_dt = datetime.utcnow()

    def __repr__(self):
        return f'Person(name={self.name}, age={self.age}, create_dt={self.create_dt})'
    
    def toJSON(self):
        return {
            'name': self.name,
            'age': self.age,
            'create_dt': self.create_dt.isoformat()
        }

In [24]:
p = Person('John', 30)

In [25]:
p

Person(name=John, age=30, create_dt=2024-12-31 06:11:51.551348)

In [26]:
p.toJSON()

{'name': 'John', 'age': 30, 'create_dt': '2024-12-31T06:11:51.551348'}

In [27]:
log_record = dict(time=datetime.utcnow(), message='Created new person record', person=p)

In [28]:
json.dumps(log_record, default=custom_json_formatter)

'{"time": "2024-12-31T06:13:16.394877", "message": "Created new person record", "person": "Person(name=John, age=30, create_dt=2024-12-31 06:11:51.551348)"}'

In [29]:
def custom_json_formatter(arg):
    if isinstance(arg, datetime):
        return arg.isoformat()
    elif isinstance(arg, set):
        return list(arg)
    elif isinstance(arg, Person):
        return arg.toJSON()
    return str(arg)

In [30]:
json.dumps(log_record, default=custom_json_formatter)

'{"time": "2024-12-31T06:13:16.394877", "message": "Created new person record", "person": {"name": "John", "age": 30, "create_dt": "2024-12-31T06:11:51.551348"}}'

In [36]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        self.create_dt = datetime.utcnow()

    def __repr__(self):
        return f'Person(name={self.name}, age={self.age}, create_dt={self.create_dt})'
    
    def toJSON(self):
        return vars(self)

In [37]:
p = Person('Python', 30)

In [38]:
log_record = dict(time=datetime.utcnow(), message='Created new person record', person=p)

In [39]:
json.dumps(log_record, default=custom_json_formatter)

'{"time": "2024-12-31T06:17:35.177419", "message": "Created new person record", "person": {"name": "Python", "age": 30, "create_dt": "2024-12-31T06:17:34.856165"}}'

In [40]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return f'Point(x={self.x}, y={self.y})'

In [41]:
def custom_json_formatter(arg):
    if isinstance(arg, datetime):
        return arg.isoformat()
    elif isinstance(arg, set):
        return list(arg)
    elif isinstance(arg, Person):
        return arg.toJSON()
    else:
        try:
            return arg.toJSON()
        except AttributeError:
            try:
                return vars(arg)
            except TypeError:
                return str(arg)


In [49]:
from decimal import Decimal
pt1 = Point(10, 20)
pt2 = Point(Decimal(10), Decimal(20.5))
person = Person('Python', 30)

In [50]:
json.dumps(p, default=custom_json_formatter)

'{"x": 10, "y": 20}'

In [55]:
from pprint import pprint
log_record = dict(time=datetime.utcnow(), message='Created new point record', point1=pt1, point2=pt2, creator=person)
pprint(json.dumps(log_record, default=custom_json_formatter))



('{"time": "2024-12-31T06:24:00.162526", "message": "Created new point '
 'record", "point1": {"x": 10, "y": 20}, "point2": {"x": "10", "y": "20.5"}, '
 '"creator": {"name": "Python", "age": 30, "create_dt": '
 '"2024-12-31T06:23:06.136662"}}')


In [56]:
from functools import singledispatch

In [58]:
@singledispatch
def json_format(arg):
    try:
        print('trying to use toJSON; arg is a', type(arg))
        return arg.toJSON()
    except AttributeError:
        try:
            print('trying to use vars; arg is a', type(arg))
            return vars(arg)
        except:
            print('trying to use str; arg is a', type(arg))
            return str(arg)

In [59]:
@json_format.register(datetime)
def _(arg):
    return arg.isoformat()

@json_format.register(set)
def _(arg):
    return list(arg)

In [60]:
log_record = dict(time=datetime.utcnow(), message='Created new point record', point1=pt1, creator=person)

In [61]:
json.dumps(log_record, default=json_format)

trying to use toJSON; arg is a <class '__main__.Point'>
trying to use vars; arg is a <class '__main__.Point'>
trying to use toJSON; arg is a <class '__main__.Person'>


'{"time": "2024-12-31T06:28:05.535350", "message": "Created new point record", "point1": {"x": 10, "y": 20}, "creator": {"name": "Python", "age": 30, "create_dt": "2024-12-31T06:23:06.136662"}}'

In [62]:
from decimal import Decimal
from fractions import Fraction

In [63]:
d = dict(a=1+1j, b=Decimal(10), c=Fraction(1, 3))
json.dumps(d, default=json_format)

trying to use toJSON; arg is a <class 'complex'>
trying to use vars; arg is a <class 'complex'>
trying to use str; arg is a <class 'complex'>
trying to use toJSON; arg is a <class 'decimal.Decimal'>
trying to use vars; arg is a <class 'decimal.Decimal'>
trying to use str; arg is a <class 'decimal.Decimal'>
trying to use toJSON; arg is a <class 'fractions.Fraction'>
trying to use vars; arg is a <class 'fractions.Fraction'>
trying to use str; arg is a <class 'fractions.Fraction'>


'{"a": "(1+1j)", "b": "10", "c": "1/3"}'

In [64]:
d = dict(a=1+1j, b=Decimal(10), c=Fraction(1, 3), d=datetime.utcnow(), p=Person('Python', 30))

In [65]:
json.dumps(d, default=json_format)

trying to use toJSON; arg is a <class 'complex'>
trying to use vars; arg is a <class 'complex'>
trying to use str; arg is a <class 'complex'>
trying to use toJSON; arg is a <class 'decimal.Decimal'>
trying to use vars; arg is a <class 'decimal.Decimal'>
trying to use str; arg is a <class 'decimal.Decimal'>
trying to use toJSON; arg is a <class 'fractions.Fraction'>
trying to use vars; arg is a <class 'fractions.Fraction'>
trying to use str; arg is a <class 'fractions.Fraction'>
trying to use toJSON; arg is a <class '__main__.Person'>


'{"a": "(1+1j)", "b": "10", "c": "1/3", "d": "2024-12-31T06:30:42.456481", "p": {"name": "Python", "age": 30, "create_dt": "2024-12-31T06:30:42.456495"}}'

In [66]:
@json_format.register(Decimal)
def _(arg):
    return f'Decimal({str(arg)})'

json.dumps(d, default=json_format)

trying to use toJSON; arg is a <class 'complex'>
trying to use vars; arg is a <class 'complex'>
trying to use str; arg is a <class 'complex'>
trying to use toJSON; arg is a <class 'fractions.Fraction'>
trying to use vars; arg is a <class 'fractions.Fraction'>
trying to use str; arg is a <class 'fractions.Fraction'>
trying to use toJSON; arg is a <class '__main__.Person'>


'{"a": "(1+1j)", "b": "Decimal(10)", "c": "1/3", "d": "2024-12-31T06:30:42.456481", "p": {"name": "Python", "age": 30, "create_dt": "2024-12-31T06:30:42.456495"}}'